In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [94]:
import tensorflow as tf 
from tensorflow.keras.applications.efficientnet import preprocess_input

directory = '/kaggle/input/abdous-egg-experiment/dataset/dataset/day_'
target_size = (224, 224)

# initialize lists to store images and labels
images = []
labels = []

# iterate over each file in the directory
for i in range(4):
    for filename in os.listdir(directory+str(i+1)):
        if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg') :
            # load and preprocess the image
            img = tf.keras.preprocessing.image.load_img(os.path.join(directory+str(i+1), filename), target_size=target_size)
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            img_array =  preprocess_input(img_array)
        
            # add the preprocessed image to the list of images
            images.append(img_array)
            labels.append(i)

# convert lists to numpy arrays
X_train = np.array(images)
Y_train = np.array(labels)

In [95]:
np.unique(Y_train)

array([0, 1, 2, 3])

In [34]:
X = all_images_array
Y = class_labels_array
Y = Y - 1

In [102]:
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
import tensorflow as tf


# Load the pre-trained ResNet50 model without the top (fully connected) layers
effnetb0 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze the weights of the pre-trained layers so they're not updated during training
for layer in effnetb0.layers:
    layer.trainable = False

# Create a new model on top of ResNet50
model = Sequential([
    effnetb0,
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(1280, activation='relu'),
    Dense(4, activation='softmax')  # 4 classes, so the output layer has 4 units with softmax activation
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',  # Since class labels are integers
              metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 7, 7, 1280)        4049571   
 )                                                               
                                                                 
 global_average_pooling2d_2  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_12 (Dense)            (None, 1280)              1639680   
                                                                 
 dense_13 (Dense)            (None, 4)                 5124      
                                                                 
Total params: 5694375 (21.72 MB)
Trainable params: 1644804 (6.27 MB)
Non-trainable params: 4049571 (15.45 MB)
_________________________________________________________________


In [103]:
model.fit(X_train,Y_train,batch_size=32,epochs=10)

Epoch 1/10


2024-02-25 04:00:03.758825: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_6/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


119/119 [==============================] - 15s 62ms/step - loss: 0.5179 - accuracy: 0.8049
Epoch 2/10
119/119 [==============================] - 7s 61ms/step - loss: 0.3306 - accuracy: 0.8768
Epoch 3/10
119/119 [==============================] - 7s 61ms/step - loss: 0.3009 - accuracy: 0.8910
Epoch 4/10
119/119 [==============================] - 7s 61ms/step - loss: 0.2474 - accuracy: 0.9105
Epoch 5/10
119/119 [==============================] - 7s 61ms/step - loss: 0.2190 - accuracy: 0.9181
Epoch 6/10
119/119 [==============================] - 7s 61ms/step - loss: 0.2013 - accuracy: 0.9192
Epoch 7/10
119/119 [==============================] - 7s 60ms/step - loss: 0.1915 - accuracy: 0.9281
Epoch 8/10
119/119 [==============================] - 7s 61ms/step - loss: 0.2015 - accuracy: 0.9231
Epoch 9/10
119/119 [==============================] - 7s 60ms/step - loss: 0.1546 - accuracy: 0.9368
Epoch 10/10
119/119 [==============================] - 7s 61ms/step - loss: 0.1536 - accuracy: 0.9439

In [107]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.applications.efficientnet import preprocess_input


# Path to the main folder containing subfolders
main_folder = '/kaggle/input/abdous-egg-experiment/dataset/test'

# List to store image arrays
image_arrays = []
names = []

# Define a target size for resizing
target_size = (224, 224)  # Adjust as needed
files =os.listdir(main_folder)
files.sort()
# Iterate through subfolders
for image_file in files:
    img = tf.keras.preprocessing.image.load_img(os.path.join(main_folder, image_file), target_size=target_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = preprocess_input(img_array)
    names.append(image_file)
    image_arrays.append(img_array)
                
print("Shape of the final image array:", len(image_arrays))
print("Shape of the final names array:", len(names))

Shape of the final image array: 419
Shape of the final names array: 419


In [108]:
predictions = model.predict(np.array(image_arrays))

14/14 [==============================] - 1s 59ms/step


In [109]:
predictionsf = np.argmax(predictions , axis=1) + 1
predictionsf

array([3, 1, 4, 4, 2, 1, 1, 3, 3, 4, 3, 1, 2, 3, 1, 4, 2, 3, 2, 3, 4, 1,
       4, 1, 4, 4, 2, 4, 1, 2, 3, 4, 2, 4, 3, 3, 1, 3, 3, 4, 1, 2, 2, 3,
       3, 4, 4, 2, 2, 1, 3, 3, 3, 1, 3, 4, 1, 3, 3, 4, 3, 4, 2, 3, 1, 1,
       4, 2, 3, 1, 3, 2, 4, 1, 4, 3, 2, 1, 3, 1, 1, 4, 1, 3, 1, 3, 2, 4,
       2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 4, 2, 4, 2, 4, 3, 2, 1, 1, 2, 4,
       3, 2, 2, 2, 4, 2, 1, 2, 2, 2, 2, 3, 4, 4, 1, 4, 2, 2, 4, 1, 4, 2,
       3, 2, 2, 4, 1, 4, 3, 2, 4, 3, 4, 2, 3, 1, 4, 1, 2, 4, 1, 2, 4, 3,
       3, 3, 3, 3, 1, 3, 2, 2, 1, 3, 1, 2, 2, 1, 4, 2, 1, 2, 3, 3, 3, 1,
       3, 1, 1, 3, 1, 4, 4, 3, 4, 4, 4, 3, 2, 4, 3, 4, 2, 1, 2, 3, 2, 2,
       2, 2, 2, 3, 4, 2, 4, 2, 4, 4, 1, 2, 2, 1, 3, 2, 2, 2, 3, 1, 1, 3,
       2, 4, 2, 4, 4, 2, 2, 1, 1, 1, 4, 2, 3, 3, 2, 2, 4, 4, 1, 2, 3, 4,
       4, 1, 1, 2, 3, 2, 4, 2, 4, 1, 4, 2, 1, 2, 1, 1, 2, 4, 2, 2, 4, 4,
       3, 2, 2, 3, 2, 2, 2, 2, 2, 3, 4, 1, 1, 2, 4, 1, 1, 4, 4, 1, 1, 2,
       3, 3, 3, 1, 2, 4, 1, 1, 4, 4, 3, 4, 3, 1, 2,

In [110]:
predictions_df = pd.DataFrame({'ID' : np.array(names), 'Target': predictionsf})
predictions_df

,ID,Target
0,00245649-f928-4878-9348-72a3e3a491ce.jpg,3
1,00f07b6d-cf27-4c8e-a91a-4c6a70f67139.jpg,1
2,021fdcac-1c0d-4162-a702-38ce19dda747.jpg,4
3,02b17e8f-67b5-408e-80f9-68312c218bda.jpg,4
4,02d23e88-9e5b-49b8-9253-b6bf5d5701ca.jpg,2
...,...,...
414,fdc9c6c8-f60c-4691-b120-6fe5a342a8ec.jpg,3
415,fdfabeda-d78d-4c6f-bdce-00ee4c70972f.jpg,3
416,fe35d9c4-10c5-4053-b326-a0eb7c6e60d9.jpg,1
417,ff9259b3-cf45-4dfc-bdd6-13e0b1df9abb.jpg,1


In [111]:
predictions_df.to_csv('pb3v2.csv',index=False)